# Clima en los cultivos

Uno de los factores principales a considerar durante las cosechas es el clima de la región, está determina las temporadas de cosecha y los tipos de cultivos que pueden existir. Por lo tanto será importante obtener la información metereológica del pais. 

## Extracción de datos

### Consistencia de la nueva información

Pretendemos obtener información del clima del país, sin embargo al observar la información obtenida del SIAP, vemos que la información esta dada según los municipios, por lo tanto debemos procurar que la información climática provenga de cada municipio. 

### Fuente
La información será extraída de 'Historical Weather API', que se puede encontrar en el siguiente elnace: https://open-meteo.com/en/docs/historical-weather-api#latitude=29.0881&longitude=-111.0103&start_date=2020-01-01&hourly=temperature_2m,relativehumidity_2m,precipitation. 

Lo principal de la API, es que proporciona información según las coordenadas geograficas de la región (latitud, longitud). Por lo tanto deberemos extraer las coordendadas de cada municipio.

### Proceso.

Esto lo haremos, mediante la extracción de información del siguiente enlace. 'https://www.coordenadas.com.es/mexico/index.php'. Para ello usaremos algunas librerías.

In [130]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import math

Primero, obtendremos la lista de Estados y el número de municipios. Esto como referencia para más adelante.

In [111]:
url_ref = 'https://diaonia.com/cuantos-municipios-hay-en-cada-estado-de-la-republica-mexicana/'
estados_ref = pd.read_html(url_ref)
estados = pd.DataFrame(estados_ref[0])
estados.head()

,Estado,Cantidad de municipios
0,Aguascalientes,11
1,Baja California,5
2,Baja California Sur,5
3,Campeche,11
4,Chiapas,124


Ahora si, comenzamos con la extracción. Esto lo haremos mediante dos métodos.
#### Directo

El paso directo, consta en usar la 'coordenadas.com.es', una página que posee las coordendadas de muchas partes del mundo, entre ellas México, por lo tanto nos interesa. Si observa el enlace a continuación https://www.coordenadas.com.es/mexico/index.php, podrá ver que aparecen todos los estados de México, entonces la usaremos para automatizar la extracción de los municipios por estado.

In [112]:
coor_url = 'https://www.coordenadas.com.es/mexico/index.php'
html = requests.get(coor_url)
print(html)

<Response [200]>


Vemos que `requests` pudo conectar con la página. Entonces usaremos `BeautifulSoup` para revisar su contenido. 

In [113]:
html_content = html.text
soup = BeautifulSoup(html_content)

# Checando el código fuente de la página principal. Vemos que los enlaces que buscamos, se encuentra en cierta clase de HTML.
data = soup.find_all('a',class_= 'list-group-item')
url_estados = []
name_estados = []
for link in data:
    lhref = link.get('href')
    ltext = link.text
    url_estados.append(lhref)
    name_estados.append(ltext)

print(url_estados, name_estados)

i_cdmx = name_estados.index('Distrito-Federal')
name_estados[i_cdmx] = 'Ciudad de México'
name_estados.sort()

['https://www.coordenadas.com.es/mexico/pueblos-de-aguascalientes/1/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-baja-california/2/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-baja-california-sur/3/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-campeche/4/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-chiapas/5/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-chihuahua/6/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-coahuila-de-zaragoza/7/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-colima/8/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-distrito-federal/9/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-durango/10/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-guanajuato/11/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-guerrero/12/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-hidalgo/13/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-jalisco/14/1', 'https://www.coordenadas.com.es/mexico/pueb

Entonces, podríamos generar un dataframe con los estados, su nombre clave y la url correspondiente.

In [114]:
estados['Nombre'] = name_estados
estados['Url'] = url_estados

estados.columns

Index(['Estado', 'Cantidad de municipios', 'Nombre', 'Url'], dtype='object')

Entonces a partir del enlace principal, obtuvimos los enlaces para obtener los enlances de los estados para obtener las coordenadas de los municipios. Ahora definimos un buscador mediante la clave para obtener la url, tomaremos como ejemplo a Sonora.

In [115]:
def info_estado(estado,columna):
    i_e = list(estados['Estado']).index(estado)
    if i_e > 0:
        if columna == '*':
            data = estados.loc[int(i_e),:]
        else:
            data = estados.loc[int(i_e),columna]
    else: 
        data = 'No se encontro el estado'
    return data

info_estado('Sonora','Url')

'https://www.coordenadas.com.es/mexico/pueblos-de-sonora/26/1'

Ahora, obtenemos las coordenadas de los municipios.

In [116]:
turl = info_estado('Sonora','Url')
thtml = pd.read_html(turl)
tdf = pd.DataFrame(thtml[0])
tdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Ciudad       60 non-null     object
 1   Coordenadas  60 non-null     object
dtypes: object(2)
memory usage: 1.1+ KB


Obtuvimos las coordenadas de los municipios de Sonora, sin embargo, como se puede apreciar. Faltan municipios. 

In [117]:
print(f'Numero de renglones en Sonora: {tdf.shape[0]}\nNúmero de Municipios de Sonora: {info_estado("Sonora","Cantidad de municipios")}')

Numero de renglones en Sonora: 60
Número de Municipios de Sonora: 72


Revisando la página de los municipios de Sonora podemos ver que se encuentran desplegados en varias pestañas. Cada pestaña puede mostrar máximo 60 municipios. Entonces debemos encontrar los que faltan, La página mostrada esta representada por él último número del enlace. Entonces nos tocará modificarlo. Para ello considere la siguiente función.

In [118]:
def modify_tabpage(url,tab):
    url = url[:-1] + str(tab)
    return url

Ahora si podemos extraer todos los municipios de Sonora

In [125]:
n_s = tdf.shape[0]

n_url = modify_tabpage(turl, 2)

html_son = pd.read_html(n_url)
Sonora2 = pd.DataFrame(html_son[0])
S = pd.concat((tdf,Sonora2),ignore_index=True)
S.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Ciudad       64 non-null     object
 1   Coordenadas  64 non-null     object
dtypes: object(2)
memory usage: 1.1+ KB


Pero aún así faltan municipios, este es el principal problema del primer método. Entonces vamos a concluir con la extracción de todas las coordendas DISPONIBLES en el url.

In [157]:
def simple_df(url):
    html_zero = pd.read_html(url)
    df = pd.DataFrame(html_zero[0])
    return df

def tabs_df(estado):
    n_muns = info_estado(estado,'Cantidad de municipios')
    url_zero = info_estado(estado,'Url')
    df = simple_df(url_zero)
    if n_muns > 60:
        max_tabs = math.ceil(n_muns / 60)
        for i in np.arange(2, max_tabs + 1):
            n_url = modify_tabpage(url_zero,i)
            n_df = simple_df(n_url)
            df = pd.concat((df, n_df),ignore_index= True)
    df['Estado'] = [estado for i in range(df.shape[0])]
    df = df[['Estado','Ciudad','Coordenadas']]
    return df

def mexico_muns_coords(selector):
    if selector == '*':
        selector = list(estados['Estado'])
    mex_df = pd.DataFrame()
    for estado in selector:
        est_df = tabs_df(estado)
        mex_df = pd.concat((mex_df,est_df),ignore_index=True)
    return mex_df

aguas = info_estado('Aguascalientes','Url')
aguas


'No se encontro el estado'

In [ ]:
url_base = "https://cuentame.inegi.org.mx/monografias/informacion/"
state = 'son'
url_final = "/territorio/div_municipal.aspx"

url_mun = url_base + state + url_final

mun_html = pd.read_html(url_mun)
municipios = pd.DataFrame(mun_html[3])[['Municipio']]
municipios = municipios.drop(municipios.index[-1])
print(municipios)

s = Sonora['Ciudad']

print(s)


                        Municipio
0                         Aconchi
1                     Agua Prieta
2                          Alamos
3                           Altar
4                        Arivechi
..                            ...
67                Villa Pesqueira
68                         Yécora
69  General Plutarco Elías Calles
70                  Benito Juárez
71         San Ignacio Río Muerto

[72 rows x 1 columns]
0                           - Aconchi
1                       - Agua-Prieta
2                            - Alamos
3                             - Altar
4                          - Arivechi
5                            - Arizpe
6                              - Atil
7                      - Bacadehuachi
8                          - Bacanora
9                           - Bacerac
10                         - Bacoachi
11                            - Bacum
12                        - Banamichi
13                        - Baviacora
14                          - Bavispe

Con esto logramos extraer los datos requeridos

In [ ]:

def get_df_mun(url):
    html = pd.read_html(url)
    df = pd.DataFrame(html[0])
    return df